In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
import datetime
import mysql.connector


# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='13.211.174.98',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hanagift',   # MySQL 사용자 비밀번호
    database='db'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()

# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 4. "기간환율변동" 선택 클릭
try:
    period_exchange_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='inqType_p'].radioForm")))
    period_exchange_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='inqType_p']\").click();")

# 오늘 날짜와 일주일 전 날짜 계산
today = datetime.date.today()
one_week_ago = today - datetime.timedelta(days=4)

# 날짜를 'YYYY-MM-DD' 형식으로 포맷
start_date = one_week_ago.strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")

# 날짜 입력
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt_p")))
date_input.clear()
date_input.send_keys(start_date)

date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqEndDt_p")))
date_input.clear()
date_input.send_keys(end_date)

# 통화 코드 리스트
currency_codes = ['AUD', 'BRL', 'CAD', 'CNY', 'EUR', 'GBP', 'JPY', 'MYR', 'RUB', 'SEK', 'USD', 'VND']

# 각 통화에 대해 데이터를 조회하고 MySQL에 삽입
for currency in currency_codes:
    print(f"Processing currency: {currency}")
    
    # 6. 통화 선택
    currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
    select = Select(currency_select)
    select.select_by_value(currency)
    
    # 7. "조회" 버튼 클릭
    query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
    query_button.click()

    # 8. 조회 후 대기 (필요에 따라 시간 조정 가능)
    time.sleep(5)

    # 데이터 리스트 초기화
    data_list = []
    
    # 9. 테이블 데이터 크롤링
    rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        data = [column.text for column in columns]
        
        # 첫 번째 값 제외, 나머지를 float로 변환
        for i in range(1, len(data)):
            try:
                data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
            except ValueError:
                print(f"변환 실패: {data[i]}")
        
        data.append(currency)  # 마지막에 통화 코드 추가
        data_list.append(data)
    
    # 10. MySQL에 데이터를 일괄 삽입
    insert_query = """
    INSERT INTO ExchangeRatePeriod (exchangeDate, buyCash, sellCash, sendRemit, receiveRemit, buyForeignCheck, exchangeRate, previousComparison, rateGap, usdConversionRate, currencyCode)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    try:
        # 데이터 삽입 실행
        for data in data_list:
            cursor.execute(insert_query, (
                data[0],  # exchangeDate
                data[1],  # buyCash
                data[2],  # sellCash
                data[3],  # sendRemit
                data[4],  # receiveRemit
                data[5],  # buyForeignCheck
                data[6],  # exchangeRate
                data[7],  # previousComparison
                data[8],  # rateGap
                data[9],  # usdConversionRate
                data[10]  # currencyCode (통화코드)
            ))

        # 트랜잭션 커밋
        conn.commit()
        print(f"{cursor.rowcount} rows inserted successfully for {currency}.")

    except mysql.connector.Error as err:
        print(f"Error for {currency}: {err}")
        conn.rollback()  # 오류 발생 시 롤백

# 마지막에 커서와 연결 종료
cursor.close()
conn.close()
driver.quit()


Processing currency: AUD
1 rows inserted successfully for AUD.
Processing currency: BRL
1 rows inserted successfully for BRL.
Processing currency: CAD
1 rows inserted successfully for CAD.
Processing currency: CNY
1 rows inserted successfully for CNY.
Processing currency: EUR
1 rows inserted successfully for EUR.
Processing currency: GBP
1 rows inserted successfully for GBP.
Processing currency: JPY
1 rows inserted successfully for JPY.
Processing currency: MYR
1 rows inserted successfully for MYR.
Processing currency: RUB
1 rows inserted successfully for RUB.
Processing currency: SEK
1 rows inserted successfully for SEK.
Processing currency: USD
1 rows inserted successfully for USD.
Processing currency: VND
1 rows inserted successfully for VND.
